In [86]:
import numpy as np 
import pandas as pd
import csv 
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
import os 
os.chdir("/Users/susanchen/Documents/GitHub/skincare_classification/Plots")

## Building Multi-label Classifiers 
### Classifiers to test: Binary Relevance (no correlation), Binary Relevance (with correlation), Multi-label Random Forest, Multi-label Decision Tree, Classifier Chain, Binary Releveance with Stacking Aggregation, Cross-Coupling Aggregation (to combat class-imbalances)

In [87]:
data = pd.read_csv('/Users/susanchen/Documents/GitHub/skincare_classification/Data_sets/Master_cleaned.csv')

In [88]:
data

,Product,Brand,Ingredients,Price,Skin_Type,active_ingredient,inactive_ingredient,active_ingredient_list,inactive_ingredient_list,num_inactive_ingredients,...,num_of_Emollients,num_of_Hydration,num_of_Skin-Restoring,num_of_Plant Extracts,num_of_Preservatives,num_of_Skin-Softening,num_of_Sensitizing,num_of_Skin-Replenishing,top_3,top3_category_list
0,#InstantDetox Facial Mask,Biobelle,"Water, Butylene Glycol, Glycerin, Trehalose, H...",3.99,['Oily'],NaN,"Water, Butylene Glycol, Glycerin, Trehalose, H...",[],"['Water', 'Butylene Glycol', 'Glycerin', 'Treh...",23,...,0,0,0,0,0,0,0,1,"['Butylene Glycol', 'Glycerin', 'Trehalose']","['Texture Enhancer', 'Skin-Replenishing, Skin-..."
1,#Peachy Facial Mask,Biobelle,"Water, Methylpropanediol, Butylene Glycol, Gly...",3.99,['Dry'],NaN,"Water, Methylpropanediol, Butylene Glycol, Gly...",[],"['Water', 'Methylpropanediol', 'Butylene Glyco...",20,...,0,0,0,0,0,0,0,1,"['Methylpropanediol', 'Butylene Glycol', 'Glyc...","[None, 'Texture Enhancer', 'Skin-Replenishing,..."
2,#Rise&Shine Facial Mask,Biobelle,"Water, Glycerin, Butylene Glycol, Triethylhexa...",3.99,['Combination'],NaN,"Water, Glycerin, Butylene Glycol, Triethylhexa...",[],"['Water', 'Glycerin', 'Butylene Glycol', 'Trie...",33,...,0,0,0,0,0,0,0,1,"['Glycerin', 'Butylene Glycol', 'Triethylhexan...","['Skin-Replenishing, Skin-Restoring', 'Texture..."
3,#RoséAllDay Facial Mask,Biobelle,"Water, Methylpropanediol, Glycerin, Propanedio...",3.99,['Combination'],NaN,"Water, Methylpropanediol, Glycerin, Propanedio...",[],"['Water', 'Methylpropanediol', 'Glycerin', 'Pr...",31,...,0,0,0,1,2,0,0,2,"['Methylpropanediol', 'Glycerin', 'Propanediol']","[None, 'Skin-Replenishing, Skin-Restoring', None]"
4,#VitaminSea Facial Mask,Biobelle,"Water, Butylene Glycol, Glycerin, Hydroxyaceto...",3.99,['Dry'],NaN,"Water, Butylene Glycol, Glycerin, Hydroxyaceto...",[],"['Water', 'Butylene Glycol', 'Glycerin', 'Hydr...",24,...,0,0,0,1,0,0,0,1,"['Butylene Glycol', 'Glycerin', 'Hydroxyacetop...","['Texture Enhancer', 'Skin-Replenishing, Skin-..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019,Youthful Vitamin C Fresh Radiance Essence,No7,"Aqua (Water), Butylene Glycol, Glycerin, Gluco...",24.99,"['Dry', 'Oily', 'Combination', 'Normal']",NaN,"Aqua (Water), Butylene Glycol, Glycerin, Gluco...",[],"['Aqua (Water)', 'Butylene Glycol', 'Glycerin'...",18,...,0,0,0,2,3,0,0,2,"['Butylene Glycol', 'Glycerin', 'Gluconolactone']","['Texture Enhancer', 'Skin-Replenishing, Skin-..."
2020,Yuza Sorbet Day Cream,Erborian,"Aqua/Water, Cyclomethicone, Glycerin, Nylon-12...",48.00,"['Oily', 'Normal', 'Sensitive', 'Combination',...",NaN,"Aqua/Water, Cyclomethicone, Glycerin, Nylon-12...",[],"['Aqua/Water', 'Cyclomethicone', 'Glycerin', '...",33,...,1,0,0,1,1,0,0,1,"['Cyclomethicone', 'Glycerin', 'Nylon-12']","['Emollients', 'Skin-Replenishing, Skin-Restor..."
2021,Yuza Sorbet Night Treatment,Erborian,"Aqua/Water, Cyclomethicone, Glycerin, Cetearyl...",55.00,"['Oily', 'Normal', 'Sensitive', 'Combination',...",NaN,"Aqua/Water, Cyclomethicone, Glycerin, Cetearyl...",[],"['Aqua/Water', 'Cyclomethicone', 'Glycerin', '...",15,...,2,0,0,0,0,0,0,1,"['Cyclomethicone', 'Glycerin', 'Cetearyl Alcoh...","['Emollients', 'Skin-Replenishing, Skin-Restor..."
2022,Yuzu Overnight Moisture Mask,Earth Therapeutics,"Water (Aqua), Propanediol, Glycerin, Hydrogena...",7.00,"['Dry', 'Normal', 'Sensitive', 'Combination']",NaN,"Water (Aqua), Propanediol, Glycerin, Hydrogena...",[],"['Water (Aqua)', 'Propanediol', 'Glycerin', 'H...",41,...,2,0,0,1,0,0,0,1,"['Propanediol', 'Glycerin', 'Hydrogenated Poly...","[None, 'Skin-Replenishing, Skin-Restoring', None]"


In [84]:
from sklearn.model_selection import train_test_split

X = data[data.columns[19:28]].values 
#y = data[data.columns[5:10]].values
y = data.Skin_Type
#y = MultiLabelBinarizer().fit_transform(y)

X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, shuffle=True, test_size=0.4, random_state = 1024)
X_val, X_test, y_val, y_test = train_test_split(X_holdout, y_holdout, shuffle=False, test_size=0.5, random_state = 1024)

print("Dataset sizes:\n\tTrain %s\n\tVal %s\n\tTest %s" % (X_train.shape, X_val.shape,X_test.shape))

Dataset sizes:
	Train (1214, 6)
	Val (405, 6)
	Test (405, 6)


In [85]:
for i in y:
    print(i)

['Oily']
['Dry']
['Combination']
['Combination']
['Dry']
['Oily', 'Normal', 'Sensitive', 'Combination', 'Dry']
['Oily', 'Normal', 'Sensitive', 'Combination', 'Dry']
['Dry', 'Oily', 'Combination', 'Normal']
['Dry', 'Oily', 'Combination', 'Normal']
['Dry', 'Oily', 'Combination', 'Normal']
['Oily', 'Normal', 'Sensitive', 'Combination', 'Dry']
['Dry']
['Dry']
['Dry']
['Oily', 'Normal', 'Sensitive', 'Combination', 'Dry']
['Oily', 'Normal', 'Sensitive', 'Combination', 'Dry']
['Dry', 'Sensitive']
['Sensitive']
['Oily', 'Combination']
['Oily', 'Normal', 'Sensitive', 'Combination', 'Dry']
['Combination']
['Combination']
['Oily', 'Normal', 'Sensitive', 'Combination', 'Dry']
['Oily', 'Normal', 'Sensitive', 'Combination', 'Dry']
['Oily', 'Normal', 'Sensitive', 'Combination', 'Dry']
['Oily', 'Normal', 'Sensitive', 'Combination', 'Dry']
['Normal', 'Oily', 'Combination']
['Oily', 'Normal', 'Sensitive', 'Combination', 'Dry']
['Dry', 'Oily', 'Combination', 'Normal']
['Dry', 'Oily', 'Combination', 'Norm

In [69]:
 from sklearn.preprocessing import MultiLabelBinarizer
 MultiLabelBinarizer().fit_transform(y)

array([[0, 1],
       [0, 1],
       [0, 1],
       ...,
       [1, 1],
       [1, 1],
       [1, 1]])

In [73]:
y_test =np.array(y_test)
y_test

array([[0, 1],
       [1, 1],
       [1, 1],
       [0, 1],
       [1, 1],
       [0, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [0, 1],
       [1, 1],
       [1, 1],
       [0, 1],
       [0, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [0, 1],
       [0, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [0, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [0, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [0, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [0, 1],
       [1, 1],
       [0, 1],
       [0, 1],
       [1, 1],
       [0, 1],
       [0, 1],
       [1, 1],
       [0, 1],
       [0, 1],
       [1, 1],
       [0, 1],
       [1, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [1, 1],
       [0, 1],
       [1, 1],
       [1, 1],
       [0, 1],
       [1, 1],
       [0, 1],
       [0, 1],
       [0,

In [8]:
pip install scikit-multilearn

     |████████████████████████████████| 89 kB 2.2 MB/s 
Note: you may need to restart the kernel to use updated packages.


In [61]:
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
# initialize binary relevance multi-label classifier with a gaussian naive bayes base classifier
classifier = BinaryRelevance(GaussianNB())
# train
classifier.fit(X_train, y_train)
# predict
predictions = classifier.predict(X_val)
# accuracy
print("Accuracy = ",accuracy_score(y_val,predictions))


Accuracy =  0.18872549019607843


In [52]:
print(predictions)

  (0, 0)	1
  (1, 0)	1
  (2, 0)	1
  (3, 0)	1
  (4, 0)	1
  (5, 0)	1
  (6, 0)	1
  (7, 0)	1
  (8, 0)	1
  (9, 0)	1
  (10, 0)	1
  (11, 0)	1
  (12, 0)	1
  (13, 0)	1
  (14, 0)	1
  (15, 0)	1
  (16, 0)	1
  (17, 0)	1
  (18, 0)	1
  (19, 0)	1
  (20, 0)	1
  (21, 0)	1
  (22, 0)	1
  (23, 0)	1
  (24, 0)	1
  :	:
  (363, 4)	1
  (365, 4)	1
  (366, 4)	1
  (367, 4)	1
  (369, 4)	1
  (370, 4)	1
  (371, 4)	1
  (373, 4)	1
  (374, 4)	1
  (381, 4)	1
  (383, 4)	1
  (384, 4)	1
  (386, 4)	1
  (387, 4)	1
  (388, 4)	1
  (389, 4)	1
  (391, 4)	1
  (392, 4)	1
  (393, 4)	1
  (394, 4)	1
  (401, 4)	1
  (402, 4)	1
  (404, 4)	1
  (405, 4)	1
  (407, 4)	1


In [70]:
from sklearn.model_selection import GridSearchCV
dt =  DecisionTreeClassifier()
param_grid = {
    'max_depth' :[5,6,7,8,9],
    'max_leaf_nodes': [5,10,15]
}
gs = GridSearchCV(dt, param_grid, scoring = "accuracy", cv= 5)
gs.fit(X, y)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': [5, 6, 7, 8, 9],
                         'max_leaf_nodes': [5, 10, 15]},
             scoring='accuracy')

In [74]:
tree = dt.fit(X_train, y_train)
y_pred = tree.predict(X_test)

In [76]:
y_test

array(['11100', '11100', '11100', '01001', '11100', '11101', '01000',
       '01100', '01101', '11100', '11001', '01100', '11100', '11100',
       '01100', '11001', '01000', '01000', '01100', '11101', '11101',
       '11101', '01000', '11101', '01100', '01000', '11101', '01000',
       '11100', '01100', '01100', '01000', '01001', '01100', '11100',
       '01001', '11100', '11101', '01100', '11100', '01000', '11101',
       '01001', '01100', '01000', '11101', '11001', '01000', '11101',
       '11000', '01101', '11101', '11100', '01001', '01100', '11100',
       '01100', '11100', '11100', '11100', '11101', '11101', '01000',
       '01000', '11100', '11100', '11000', '01000', '11100', '01000',
       '11101', '11000', '11100', '11100', '01000', '11100', '01100',
       '11100', '01000', '11101', '11100', '01100', '11100', '01100',
       '01100', '11101', '11000', '11101', '01000', '11100', '11101',
       '01000', '11100', '11000', '11100', '11101', '11101', '11101',
       '11000', '111

In [78]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))


0.0


In [30]:
sorted(sklearn.metrics.SCORERS.keys())

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_root_mean_squared_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'roc_auc_ovo',
 'roc_auc_ovo_weighted',
 'roc_auc_ovr',
 'roc_auc_ovr_weighted',
 'v_measure_score']